<a href="https://colab.research.google.com/github/diegoqprobst/IVA-calculator/blob/master/Procesamiento_de_Voz_por_medio_del_modelo_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de Voz por medio del modelo Whisper


### Instalación de las depedencias

In [ ]:
! pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tuza0xv5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tuza0xv5
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Importar Bibliotecas

In [ ]:
import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm


pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

### Fleur Datasets[texto del enlace](https://)

In [ ]:
import ipywidgets as widgets

languages = {"af_za": "Afrikaans", "am_et": "Amharic", "ar_eg": "Arabic", "as_in": "Assamese", "az_az": "Azerbaijani", "be_by": "Belarusian", "bg_bg": "Bulgarian", "bn_in": "Bengali", "bs_ba": "Bosnian", "ca_es": "Catalan", "cmn_hans_cn": "Chinese", "cs_cz": "Czech", "cy_gb": "Welsh", "da_dk": "Danish", "de_de": "German", "el_gr": "Greek", "en_us": "English", "es_419": "Spanish", "et_ee": "Estonian", "fa_ir": "Persian", "fi_fi": "Finnish", "fil_ph": "Tagalog", "fr_fr": "French", "gl_es": "Galician", "gu_in": "Gujarati", "ha_ng": "Hausa", "he_il": "Hebrew", "hi_in": "Hindi", "hr_hr": "Croatian", "hu_hu": "Hungarian", "hy_am": "Armenian", "id_id": "Indonesian", "is_is": "Icelandic", "it_it": "Italian", "ja_jp": "Japanese", "jv_id": "Javanese", "ka_ge": "Georgian", "kk_kz": "Kazakh", "km_kh": "Khmer", "kn_in": "Kannada", "ko_kr": "Korean", "lb_lu": "Luxembourgish", "ln_cd": "Lingala", "lo_la": "Lao", "lt_lt": "Lithuanian", "lv_lv": "Latvian", "mi_nz": "Maori", "mk_mk": "Macedonian", "ml_in": "Malayalam", "mn_mn": "Mongolian", "mr_in": "Marathi", "ms_my": "Malay", "mt_mt": "Maltese", "my_mm": "Myanmar", "nb_no": "Norwegian", "ne_np": "Nepali", "nl_nl": "Dutch", "oc_fr": "Occitan", "pa_in": "Punjabi", "pl_pl": "Polish", "ps_af": "Pashto", "pt_br": "Portuguese", "ro_ro": "Romanian", "ru_ru": "Russian", "sd_in": "Sindhi", "sk_sk": "Slovak", "sl_si": "Slovenian", "sn_zw": "Shona", "so_so": "Somali", "sr_rs": "Serbian", "sv_se": "Swedish", "sw_ke": "Swahili", "ta_in": "Tamil", "te_in": "Telugu", "tg_tj": "Tajik", "th_th": "Thai", "tr_tr": "Turkish", "uk_ua": "Ukrainian", "ur_pk": "Urdu", "uz_uz": "Uzbek", "vi_vn": "Vietnamese", "yo_ng": "Yoruba"}
selection = widgets.Dropdown(
    options=[("Select language", None), ("----------", None)] + sorted([(f"{v} ({k})", k) for k, v in languages.items()]),
    value="ko_kr",
    description='Language:',
    disabled=False,
)

selection


Dropdown(description='Language:', index=39, options=(('Select language', None), ('----------', None), ('Afrika…

In [ ]:
lang = selection.value
language = languages[lang]

assert lang is not None, "Please select a language"
print(f"Selected language: {language} ({lang})")

Selected language: Spanish (es_419)


In [ ]:
def download(url: str, target_path: str):
    with urllib.request.urlopen(url) as source, open(target_path, "wb") as output:
        with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit='iB', unit_scale=True, unit_divisor=1024) as loop:
            while True:
                buffer = source.read(8192)
                if not buffer:
                    break

                output.write(buffer)
                loop.update(len(buffer))


class Fleurs(torch.utils.data.Dataset):
    """
    A simple class to wrap Fleurs and subsample a portion of the dataset as needed.
    """
    def __init__(self, lang, split="test", subsample_rate=1, device=DEVICE):
        url = f"https://storage.googleapis.com/xtreme_translations/FLEURS102/{lang}.tar.gz"
        tar_path = os.path.expanduser(f"~/.cache/fleurs/{lang}.tgz")
        os.makedirs(os.path.dirname(tar_path), exist_ok=True)

        if not os.path.exists(tar_path):
            download(url, tar_path)

        all_audio = {}
        with tarfile.open(tar_path, "r:gz") as tar:
            for member in tar.getmembers():
                name = member.name
                if name.endswith(f"{split}.tsv"):
                    labels = pd.read_table(tar.extractfile(member), names=("id", "file_name", "raw_transcription", "transcription", "_", "num_samples", "gender"))

                if f"/{split}/" in name and name.endswith(".wav"):
                    audio_bytes = tar.extractfile(member).read()
                    all_audio[os.path.basename(name)] = wavfile.read(io.BytesIO(audio_bytes))[1]

        self.labels = labels.to_dict("records")[::subsample_rate]
        self.all_audio = all_audio
        self.device = device

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        record = self.labels[item]
        audio = torch.from_numpy(self.all_audio[record["file_name"]].copy())
        text = record["transcription"]

        return (audio, text)

In [ ]:
dataset = Fleurs(lang, subsample_rate=10)  # subsample 10% of the dataset for a quick demo


  0%|                                              | 0.00/2.14G [00:00<?, ?iB/s]

In [ ]:
from ipywidgets import FileUpload
import io
import librosa

# Crear un widget de carga de archivos
file_upload = FileUpload()
display(file_upload)

# Esperar a que el usuario cargue el archivo
while not file_upload.value:
    pass

# Obtener el contenido del archivo cargado
audio_content = io.BytesIO(file_upload.value[list(file_upload.value.keys())[0]]["content"])

# Cargar el audio utilizando librosa
audio, sr = librosa.load(audio_content, sr=None, mono=True)

# Luego, puedes pasar este audio a tu modelo para realizar la inferencia


FileUpload(value={}, description='Upload')

# Correr el código

In [ ]:
model = whisper.load_model("medium")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|█████████████████████████████████████| 1.42G/1.42G [00:23<00:00, 64.5MiB/s]


Model is multilingual and has 762,321,920 parameters.


In [ ]:
options = dict(language=language, beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)
translate_options = dict(task="translate", **options)

In [ ]:
references = []
transcriptions = []
translations = []

for audio, text in tqdm(dataset):
    transcription = model.transcribe(audio, **transcribe_options)["text"]
    translation = model.transcribe(audio, **translate_options)["text"]

    transcriptions.append(transcription)
    translations.append(translation)
    references.append(text)

  0%|          | 0/91 [00:00<?, ?it/s]

In [ ]:
data = pd.DataFrame(dict(reference=references, transcription=transcriptions, translation=translations))
data

,reference,transcription,translation
0,esto parece tener sentido ya que en la tierra no se percibe su movimiento ¿cierto?,"Esto parece tener sentido, ya que en la Tierra no se percibe su movimiento, ¿cierto?","This seems to make sense, since its movement is not perceived on Earth, right?"
1,las escenas se proyectan en las pirámides y todas ellas son iluminadas,Las escenas se proyectan en las pirámides y todas ellas son iluminadas.,The scenes are projected in the pyramids and all of them are illuminated.
2,estas parejas podrían elegir planificar la adopción de un bebé,Estas parejas podrían elegir planificar la adopción de un bebé.,These couples could choose to plan the adoption of a baby.
3,los equipos de rescate antincendios lograron extinguir el fuego a las 11:35 p m,Los equipos de rescate antiincendios lograron extinguir el fuego a las 11.35 PM.,The anti-fire rescue teams managed to extinguish the fire at 11.35 p.m.
4,en definitiva no debería causarnos sorpresa si nuestros propios predecesores resolvían sus «problemas proteicos» del mismo modo que los chimpancés de la sabana en la actualidad,"En definitiva, no debería causarnos sorpresa si nuestros propios predecesores resolvían sus problemas proteicos del mismo modo que los chimpancés de la sabana en la actualidad.","In short, it should not surprise us if our own predecessors solved their protein problems in the same way as the chimpanzees of the savannah today."
5,según la página web de noticias de entretenimiento tmz el fotógrafo detuvo el vehículo del lado contrario del bulevar sepulveda e intentó tomar fotografías de la parada policial antes de cruzar la calle y seguir lo que causó que el agente de la patrulla caminera de california que realizaba la detención de vehículos le diera dos veces la orden de volver a cruzar,"Según la página web de noticias de entretenimiento TMZ, el fotógrafo detuvo el vehículo del lado del boulevard Sepúlveda e intentó tomar fotografías de la parada policial antes de cruzar la calle y seguir, lo que causó que la gente de la Patrulla Caminera de California que realizaba la detención de vehículos le diera dos veces la orden de volver a cruzar.","According to the web page of entertainment news TMZ, the photographer stopped the vehicle on the side of the Sepulveda boulevard and tried to take photographs of the police stop before crossing the street and continue, which caused the people of the California road patrol who held the vehicle detention to give him twice the order to return to cross."
6,todo en el universo está hecho de materia compuesta por partículas pequeñas denominadas átomos,"Todo el universo está hecho de materia, compuesta por partículas pequeñas denominadas átomos.","The whole universe is made of matter, composed of small particles called atoms."
7,finlandia es un excelente destino para navegar: la «tierra de los mil lagos» cuenta además con miles de islas tanto en los lagos como en los archipiélagos costeros,"Finlandia es un excelente destino para navegar, la tierra de los mil lagos, cuenta además con miles de islas tanto en los lagos como en los archipiélagos costeros.","Finland is an excellent destination for sailing, the land of the thousand lakes, it also has thousands of islands both in the lakes and coastal archipelagos."
8,los viajes de estudio constituyen una parte importante de todo salón de clases es muy frecuente que una docente desee llevar a sus estudiantes a sitios a los que no es posible llegar en autobús,Los viajes de estudios constituyen una parte importante de todo el salón de clases. Es muy frecuente que una docente desee llevar a sus estudiantes a sitios a los que no es posible llegar en autobús.,Study trips are an important part of all classroom. It is very frequent that a teacher wants to take his students to places where it is not possible to get by bus.
9,el anuncio se hizo tras la conversación telefónica que trump mantuvo con con el presidente de turquía recep tayyip erdoğan,"El anuncio se hizo tras la 